In [3]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import MetaTrader5 as mt5
from datetime import datetime, timedelta
from scipy.signal import argrelextrema

# ─── Machine Learning Imports ───────────────────────────────────────────────────
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
# ────────────────────────────────────────────────────────────────────────────────


# ─── User Parameters ────────────────────────────────────────────────────────────
SYMBOLS = ['EURUSD']
START_TIME = datetime(2025, 12, 17, 12)
END_TIME = datetime(2025, 12, 17, 23)

RESAMPLE_INTERVAL = '5S'
TIMEFRAMES = ['5S', '120S', '180S', '300S', '900S']
BIN_WIDTH = 0.000009
THRESHOLD_RANGE = np.arange(0.5, 100, 0.1)
VAD_MAX_WIDTH_SECONDS = 400
VAD_WIDTH = timedelta(seconds=VAD_MAX_WIDTH_SECONDS)
HIGH_LOG_RETURN_THRESHOLD = 1
VALUE_AREA_PERCENT = 0.70 # Standard 70% for Value Area

# ─── HFT / ML Parameters ────────────────────────────────────────────────────────
ROLLING_WINDOW_TICKS = 400
ML_CONTAMINATION = 0.5
ABSORPTION_STRICTNESS = 0.5
# ────────────────────────────────────────────────────────────────────────────────


# ───────────────────────────────────────────────────────────────────────────────
# VOLUME PROFILE MODULE: POC, VAH, VAL (Calculates standard VP)
# ───────────────────────────────────────────────────────────────────────────────
def calculate_volume_profile(df_window, bin_width, value_area_percent=0.70):
    """
    Calculates the Volume Profile, Point of Control (POC), Value Area High (VAH),
    and Value Area Low (VAL) for a given DataFrame window using tick volume.
    """
    if df_window.empty:
        return None, None, None, None, pd.DataFrame()

    df_window['price'] = (df_window['bid'] + df_window['ask']) / 2
    
    # Use 'real_volume' or 'tick_volume' for weights, fallback to tick count (1)
    if 'real_volume' in df_window.columns:
        volumes = df_window['real_volume'].fillna(1)
    elif 'tick_volume' in df_window.columns:
        volumes = df_window['tick_volume'].fillna(1) 
    else:
        volumes = pd.Series(1, index=df_window.index) 

    price_min, price_max = df_window['price'].min(), df_window['price'].max()
    
    price_bins = np.arange(price_min - bin_width, price_max + bin_width * 2, bin_width)
    
    hist, edges = np.histogram(df_window['price'], bins=price_bins, weights=volumes)
    
    if hist.sum() == 0:
        return None, None, None, None, pd.DataFrame()

    vp_df = pd.DataFrame({
        'volume': hist,
        'price_start': edges[:-1],
        'price_end': edges[1:]
    })
    
    # 1. Point of Control (POC)
    poc_row = vp_df.loc[vp_df['volume'].idxmax()]
    poc_price = (poc_row['price_start'] + poc_row['price_end']) / 2
    poc_volume = poc_row['volume']

    # 2. Value Area (VA)
    total_volume = vp_df['volume'].sum()
    value_area_target = total_volume * value_area_percent
    
    remaining_df = vp_df.sort_values(by='volume', ascending=False).reset_index(drop=True)
    
    accumulated_volume = 0
    value_area_bins = []
    
    for _, row in remaining_df.iterrows():
        accumulated_volume += row['volume']
        value_area_bins.append(row)
        if accumulated_volume >= value_area_target:
            break

    if not value_area_bins:
        return poc_price, poc_volume, None, None, vp_df

    va_prices = [row['price_start'] for row in value_area_bins] + [row['price_end'] for row in value_area_bins]
    
    vah = max(va_prices)
    val = min(va_prices)

    return poc_price, poc_volume, vah, val, vp_df

# ───────────────────────────────────────────────────────────────────────────────
# MACHINE LEARNING MODULE: HFT ABSORPTION DETECTION (Retained)
# ───────────────────────────────────────────────────────────────────────────────
def detect_hft_absorption(df_window):
    # ... (Original function code remains the same)
    if len(df_window) < ROLLING_WINDOW_TICKS + 10:
        return pd.Series(dtype=float)

    df_window['rolling_volatility'] = df_window['bid'].rolling(ROLLING_WINDOW_TICKS).std()

    time_deltas = df_window['time'].diff().dt.total_seconds().fillna(0)
    df_window['rolling_duration'] = time_deltas.rolling(ROLLING_WINDOW_TICKS).sum()

    valid_data = df_window.dropna(subset=['rolling_volatility', 'rolling_duration']).copy()
    if valid_data.empty:
        return pd.Series(dtype=float)

    features = valid_data[['rolling_volatility', 'rolling_duration']].values
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    iso_forest = IsolationForest(contamination=ML_CONTAMINATION, random_state=42)
    valid_data['anomaly_score'] = iso_forest.fit_predict(features_scaled)

    vol_threshold = valid_data['rolling_volatility'].quantile(ABSORPTION_STRICTNESS)
    dur_threshold = valid_data['rolling_duration'].quantile(0.4)

    hft_mask = (
        (valid_data['anomaly_score'] == -1) &
        (valid_data['rolling_volatility'] < vol_threshold) &
        (valid_data['rolling_duration'] < dur_threshold)
    )

    return valid_data[hft_mask]

# ───────────────────────────────────────────────────────────────────────────────
# HEAT BUBBLE MODULE (Retained)
# ───────────────────────────────────────────────────────────────────────────────
def make_heat_bubbles(df_window, log_ret, anomalies, side_name):
    # ... (Original function code remains the same)
    if anomalies.empty:
        return []

    prices = df_window.loc[anomalies.index, side_name]
    if prices.empty:
        return []

    σ = np.std(log_ret)
    if σ == 0:
        return []

    z_vals = (log_ret.loc[anomalies.index] / σ).abs()
    z_norm = (z_vals - z_vals.min()) / (z_vals.max() - z_vals.min() + 1e-12)

    marker_size = 6 + z_norm * 28
    fills = 0.15 + z_norm * 0.25
    fills = np.clip(fills, 0, 0.40)

    colors = []
    for ret, alpha in zip(log_ret.loc[anomalies.index], fills):
        if ret > 0:
            colors.append(f'rgba(255,70,70,{alpha:.3f})')
        else:
            colors.append(f'rgba(50,255,50,{alpha:.3f})')

    return [
        go.Scatter(
            x=df_window.loc[anomalies.index, "time"],
            y=prices,
            mode="markers",
            marker=dict(
                size=marker_size,
                color=colors,
                line=dict(color="rgba(0,0,0,0.70)", width=0.6)
            ),
            hovertemplate="<b>Anomaly</b><br>Time: %{x}<br>Price: %{y:.5f}<extra></extra>",
            showlegend=False,
            # Assign a specific name/group for easy toggling
            name='Heatmap Bubble' 
        )
    ]


# ─── MT5 Init ─────────────────────────────────────────────────────────────
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    exit()


# ─── Main Logic ──────────────────────────────────────────────────────────
for symbol in SYMBOLS:

    print(f"Downloading ticks for {symbol}...")
    ticks = mt5.copy_ticks_range(symbol, START_TIME, END_TIME, mt5.COPY_TICKS_ALL)
    if ticks is None or len(ticks) == 0:
        print(f"No data for {symbol}")
        continue

    df = pd.DataFrame(ticks)
    df['time'] = pd.to_datetime(df['time_msc'], unit='ms')
    
    cols_to_keep = ['time', 'bid', 'ask', 'tick_volume', 'real_volume', 'last_volume']
    available_cols = [col for col in cols_to_keep if col in df.columns]
    
    if 'real_volume' not in available_cols:
         if 'tick_volume' not in available_cols:
              df['tick_volume'] = 1 
              available_cols.append('tick_volume')

    df = df[available_cols].dropna(subset=['time', 'bid', 'ask'])
    df['date'] = df['time'].dt.date

    price_min, price_max = df['bid'].min(), df['bid'].max()
    bin_edges = np.arange(price_min, price_max + BIN_WIDTH, BIN_WIDTH) 

    fig = make_subplots(
        rows=1, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.03
    )
    
    # --------------------------------------------------------------------------
    # Dictionary to track trace names for toggling
    # --------------------------------------------------------------------------
    trace_groups = {
        'candlesticks': [],
        'volume_profile': [],
        'hft_activity': [],
        'heatmap_bubbles': []
    }
    
    # Candlesticks
    for tf in TIMEFRAMES:
        ohlc_df = df.set_index('time').resample(tf).agg({'bid': 'ohlc'}).dropna()
        if not ohlc_df.empty:
            ohlc_df.columns = ['open', 'high', 'low', 'close']
            ohlc_df.reset_index(inplace=True)
        else:
            ohlc_df = pd.DataFrame(columns=['time', 'open', 'high', 'low', 'close'])

        visible_flag = (tf == RESAMPLE_INTERVAL)
        trace_name = f"candles_{tf}"
        trace = go.Candlestick(
            x=ohlc_df['time'],
            open=ohlc_df['open'],
            high=ohlc_df['high'],
            low=ohlc_df['low'],
            close=ohlc_df['close'],
            increasing_line_color="#18CF00",
            decreasing_line_color="#FF2222",
            opacity=0.8,
            showlegend=False,
            visible=visible_flag,
            name=trace_name
        )
        fig.add_trace(trace, row=1, col=1)
        trace_groups['candlesticks'].append(trace_name)


    # ─── 1-Hour Processing Block ──────────────────────────────────────────
    current = df['time'].min().replace(minute=0, second=0, microsecond=0)
    end_time = df['time'].max()
    
    # Trace counter to generate unique names for VP elements
    vp_counter = 0

    print("Processing Hourly Windows with ML and Volume Profile...")

    while current < end_time:

        win_start = current
        win_end = current + timedelta(hours=1)
        window_df = df[(df['time'] >= win_start) & (df['time'] < win_end)].copy()

        if len(window_df) < ROLLING_WINDOW_TICKS + 20:
            current += timedelta(hours=1)
            continue
        
        # Increment counter for unique trace names
        vp_counter += 1
        vp_prefix = f"VP_{vp_counter}"

        # ==================================================================
        # 1. Standard Volume Profile (VP) Plotting
        # ==================================================================
        poc, poc_vol, vah, val, vp_df = calculate_volume_profile(
            window_df, BIN_WIDTH, VALUE_AREA_PERCENT
        )

        if poc is not None:
            
            # --- VP LINES (Thin, Pink/Yellow) ---
            # VAH and VAL lines (Pink, thin and dashed)
            if vah is not None and val is not None:
                # VAH line
                trace_name = f"{vp_prefix}_VAH"
                fig.add_trace(
                    go.Scatter(
                        x=[win_start, win_end], y=[vah, vah], mode="lines",
                        line=dict(color="pink", width=1, dash='dot'),
                        name=trace_name, showlegend=False, hoverinfo='skip'
                    )
                )
                trace_groups['volume_profile'].append(trace_name)
                
                # VAL line
                trace_name = f"{vp_prefix}_VAL"
                fig.add_trace(
                    go.Scatter(
                        x=[win_start, win_end], y=[val, val], mode="lines",
                        line=dict(color="pink", width=1, dash='dot'),
                        name=trace_name, showlegend=False, hoverinfo='skip'
                    )
                )
                trace_groups['volume_profile'].append(trace_name)

            # POC line (Yellow, thin)
            trace_name = f"{vp_prefix}_POC"
            fig.add_trace(
                go.Scatter(
                    x=[win_start, win_end], y=[poc, poc], mode="lines",
                    line=dict(color="yellow", width=1.5),
                    name=trace_name, showlegend=False,
                    hovertemplate=f"<b>POC ({win_start.strftime('%H:%M')})</b><br>Price: %{{y:.5f}}<br>Volume: {poc_vol:.0f}<extra></extra>",
                )
            )
            trace_groups['volume_profile'].append(trace_name)
            
            # --- VOLUME PROFILE BARS (Grey/Pink) ---
            max_volume = vp_df['volume'].max()
            for i, row in vp_df.iterrows():
                if row['volume'] > 0:
                    normalized_width = (row['volume'] / max_volume) * VAD_MAX_WIDTH_SECONDS 
                    
                    y0, y1 = row['price_start'], row['price_end']
                    
                    # FIX: Plot the profile extending to the RIGHT from win_start
                    # The profile occupies a section of the time axis starting at win_start
                    x0 = win_start
                    x1 = win_start + timedelta(seconds=normalized_width) # Extend to the right
                    
                    # Determine color: Grey for outside VA, Pink for inside VA
                    fill_color = 'rgba(128,128,128,0.2)' # Grey for non-VA
                    
                    # Check if the bin is within the Value Area
                    if vah is not None and val is not None and val <= y0 and y1 <= vah:
                        fill_color = 'rgba(255,192,203,0.3)' # Pink/Light Magenta for VA
                    
                    # Highlight POC bin
                    if abs(y0 - poc) < BIN_WIDTH * 0.51: # Use 0.51 factor to account for floating point
                        fill_color = 'rgba(255,255,0,0.5)' # Yellow for POC bin
                        
                    trace_name = f"{vp_prefix}_bar_{i}"
                    fig.add_trace(go.Scatter(
                        x=[x0, x0, x1, x1, x0],
                        y=[y0, y1, y1, y0, y0],
                        fill='toself',
                        fillcolor=fill_color,
                        line=dict(width=0),
                        mode='lines', hoverinfo='skip',
                        showlegend=False,
                        name=trace_name # Assign a unique name for toggling
                    ))
                    trace_groups['volume_profile'].append(trace_name)


        # ==================================================================
        # 2. HFT Absorption & Heat Bubbles
        # ==================================================================
        
        # HFT Absorption Detection (ML)
        hft_activity = detect_hft_absorption(window_df)

        # HFT Activity Zone Plotting
        if not hft_activity.empty:
            hft_origin_time = hft_activity['time'].min()
            hft_origin_price = hft_activity.loc[hft_activity['time'].idxmin(), 'bid']
            
            trace_name = f"HFT_Line_{vp_counter}"
            fig.add_trace(
                go.Scatter(
                    x=[hft_origin_time, end_time],
                    y=[hft_origin_price, hft_origin_price],
                    mode="lines",
                    line=dict(color="cyan", width=1),
                    name=trace_name, 
                    hovertemplate=f"<b>HFT Activity</b><br>Price: {hft_origin_price:.5f}<extra></extra>",
                    showlegend=False
                )
            )
            trace_groups['hft_activity'].append(trace_name)
            
            # The annotation itself is not a trace, but it's part of the HFT feature.
            # It will always be visible unless the user zooms out.
            fig.add_annotation(
                x=hft_origin_time,
                y=hft_origin_price,
                text="HFT activity",
                showarrow=True,
                arrowhead=2,
                ax=0,
                ay=-30,
                font=dict(color="cyan", size=12),
                align="center"
            )


        # Heat Bubbles (Big Anomalies)
        log_returns_bid = np.log(window_df['bid'] / window_df['bid'].shift(1)).dropna()
        log_returns_ask = np.log(window_df['ask'] / window_df['ask'].shift(1)).dropna()

        std_bid = np.std(log_returns_bid) if not log_returns_bid.empty else 0
        std_ask = np.std(log_returns_ask) if not log_returns_ask.empty else 0

        if std_bid > 0:
            big_anoms_bid = log_returns_bid[
                abs(log_returns_bid) > HIGH_LOG_RETURN_THRESHOLD * std_bid
            ]
        else:
            big_anoms_bid = pd.Series(dtype=float)

        if std_ask > 0:
            big_anoms_ask = log_returns_ask[
                abs(log_returns_ask) > HIGH_LOG_RETURN_THRESHOLD * std_ask
            ]
        else:
            big_anoms_ask = pd.Series(dtype=float)

        # Add Heat Bubbles and track their names
        for anoms, side_label, log_ret in [
            (big_anoms_bid, 'bid', log_returns_bid),
            (big_anoms_ask, 'ask', log_returns_ask)
        ]:
            bubble_traces = make_heat_bubbles(window_df, log_ret, anoms, side_label)
            for tr in bubble_traces:
                trace_name = f"Bubble_{side_label}_{vp_counter}"
                tr.name = trace_name # Overwrite generic name
                fig.add_trace(tr, row=1, col=1)
                trace_groups['heatmap_bubbles'].append(trace_name)

        current += timedelta(hours=1)
    
    # --------------------------------------------------------------------------
    # PLOTLY BUTTONS & VISIBILITY LOGIC
    # --------------------------------------------------------------------------
    
    # Map trace names to indices
    trace_name_to_index = {trace.name: i for i, trace in enumerate(fig.data)}
    
    # Initial Visibility State (All visible, except non-default candles)
    initial_visibility = [trace.visible for trace in fig.data]
    
    # Create toggle buttons
    toggle_buttons = []
    
    def create_toggle_button(label, group_key):
        
        # Determine the current visibility state of the group traces
        group_indices = [trace_name_to_index[name] for name in trace_groups[group_key] if name in trace_name_to_index]
        
        # Function to generate the visibility array when the button is pressed
        def get_visibility_args(should_be_visible):
            # Start with the initial visibility array
            args = list(initial_visibility)
            for index in group_indices:
                args[index] = should_be_visible
            
            # The structure for Plotly.update is [new_data, new_layout]
            return [{"visible": args}]

        return [
            dict(
                label=f"Hide {label}",
                method="restyle",
                args=get_visibility_args(False)
            ),
            dict(
                label=f"Show {label}",
                method="restyle",
                args=get_visibility_args(True)
            )
        ]

    # Add Timeframe Toggle Buttons
    candlestick_indices = [trace_name_to_index[name] for name in trace_groups['candlesticks'] if name in trace_name_to_index]
    tf_buttons = []
    for i, tf in enumerate(TIMEFRAMES):
        visible_arr = list(initial_visibility)
        for idx in candlestick_indices:
            visible_arr[idx] = False
        visible_arr[candlestick_indices[i]] = True
        tf_buttons.append(dict(label=tf, method="update", args=[{"visible": [visible_arr]}]))

    # Add Feature Toggle Buttons
    toggle_buttons.extend(create_toggle_button('Volume Profile', 'volume_profile'))
    toggle_buttons.extend(create_toggle_button('HFT Activity', 'hft_activity'))
    toggle_buttons.extend(create_toggle_button('Heatmap', 'heatmap_bubbles'))


    fig.update_layout(
        title=dict(
            text="Institutional Footprint: Volume Profile + ML Absorption + Heat Bubbles",
            font=dict(size=24, color="white")
        ),
        height=1400,
        width=2800,
        template='plotly_dark',
        plot_bgcolor="#1e1e1e",
        paper_bgcolor="#1e1e1e",
        hovermode='x unified',
        dragmode='pan',
        xaxis_rangeslider_visible=False,
        updatemenus=[
            # Timeframe Selector
            dict(
                active=TIMEFRAMES.index(RESAMPLE_INTERVAL),
                buttons=tf_buttons, 
                x=0.02, y=1.05,
                xanchor='left', yanchor='top',
                font=dict(color='white')
            ),
            # Feature Toggle Buttons
            dict(
                buttons=toggle_buttons,
                type='buttons',
                direction='right',
                pad={'r': 10, 't': 10},
                showactive=True,
                x=0.25, # Position the feature buttons to the right of TF buttons
                y=1.05,
                xanchor='left', yanchor='top',
                font=dict(color='white')
            )
        ]
    )

    fig.update_xaxes(showgrid=True, gridcolor='rgba(70,70,70,0.2)')
    fig.update_yaxes(showgrid=True, gridcolor='rgba(40,40,40,0.2)')

    print("Plot complete.")
    fig.show()

mt5.shutdown()

Processing Hourly Windows with ML and Volume Profile...


C:\Users\Asus\AppData\Local\Temp\ipykernel_4316\1041259758.py:236: FutureWarning:

'S' is deprecated and will be removed in a future version, please use 's' instead.

C:\Users\Asus\AppData\Local\Temp\ipykernel_4316\1041259758.py:236: FutureWarning:

'S' is deprecated and will be removed in a future version, please use 's' instead.

C:\Users\Asus\AppData\Local\Temp\ipykernel_4316\1041259758.py:236: FutureWarning:

'S' is deprecated and will be removed in a future version, please use 's' instead.

C:\Users\Asus\AppData\Local\Temp\ipykernel_4316\1041259758.py:236: FutureWarning:

'S' is deprecated and will be removed in a future version, please use 's' instead.

C:\Users\Asus\AppData\Local\Temp\ipykernel_4316\1041259758.py:236: FutureWarning:

'S' is deprecated and will be removed in a future version, please use 's' instead.



Plot complete.


True